In [1]:
import os
import dandelion as ddl
import pandas as pd
import numpy as np

In [2]:
os.chdir('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')

In [3]:
df = pd.read_csv('h5ad/df.fil3.kt.metadata_for_bcr.csv.gz')
df.head()

/nfs/users/nfs_k/kt16/miniconda2/envs/dandelion/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (54,58,62,74,78,82,86,145,146) have mixed types.Specify dtype option on import or set low_memory=False.


,Unnamed: 0,orig.ident,nCount_RNA,nFeature_RNA,nCount_ADT,nFeature_ADT,nCount_ADT_raw,nFeature_ADT_raw,nCount_RNA_raw,nFeature_RNA_raw,...,temp_WCC,temp_neut,temp_mono,temp_lymph,temp_CRP,temp_ddimer,temp_ferritin,temp_trop,WHO_temp_severity,individual_id
0,UK-CIC10689738+UK-CIC10690026_AAACCTGAGACTAGAT-1,UK-CIC10689738+UK-CIC10690026,2655.188894,1214,711.520058,75,1864.0,129,2693.0,1217,...,NaN,NaN,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,critical,C116
1,UK-CIC10689738+UK-CIC10690026_AAACCTGAGACTGTAA-1,UK-CIC10689738+UK-CIC10690026,6272.316400,2120,1422.908667,55,3238.0,134,6328.0,2121,...,NaN,NaN,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,critical,C116
2,UK-CIC10689738+UK-CIC10690026_AAACCTGAGGCAATTA-1,UK-CIC10689738+UK-CIC10690026,3937.575336,1515,1011.694019,77,2336.0,134,3971.0,1516,...,NaN,NaN,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,NaN,C103
3,UK-CIC10689738+UK-CIC10690026_AAACCTGAGTCAAGGC-1,UK-CIC10689738+UK-CIC10690026,5605.811179,1935,3712.024531,55,12002.0,137,5654.0,1935,...,NaN,NaN,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,critical,C116
4,UK-CIC10689738+UK-CIC10690026_AAACCTGCAACAACCT-1,UK-CIC10689738+UK-CIC10690026,3806.958234,1411,962.904441,79,2061.0,128,3842.0,1412,...,NaN,NaN,NaN,NaN,NaN,-2147483648,-2147483648,-2147483648,critical,C116


In [4]:
ddl.logging.print_header()

dandelion==0.2.2.dev107 pandas==1.2.4 numpy==1.22.4 matplotlib==3.3.4 networkx==2.5.1 scipy==1.6.2


In [5]:
# run ddl.pp.format_fastas first so that the barcodes are attached correctly
import os
os.chdir('raw/BCR')
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f
samples = list(listdir_nohidden('.'))
len(samples)

120

In [6]:
ddl.pp.format_fastas(samples, prefix = samples, filename_prefix = ['all' for x in samples], remove_trailing_hyphen_number = False)

Formating fasta(s) : 100%|██████████| 120/120 [23:18<00:00, 11.65s/it]


In [7]:
# generate a dictionary where each individual's cell id is assigned to a list-in-dict
individuals = df.groupby('individual_id')['Unnamed: 0'].apply(list).to_dict()

In [8]:
from dandelion.utilities import fasta_iterator, Write_output

In [9]:
# create new fastas in a separate folder corresponding to each individual id
seqs = {}
for samp in samples:
    fh = open(samp + '/dandelion/all_contig.fasta', 'r')
    for header, sequence in fasta_iterator(fh):
        seqs[header] = sequence
    fh.close()

In [10]:
annot = []
for samp in samples:
    annot.append(pd.read_csv(samp + '/dandelion/all_contig_annotations.csv'))
annot = pd.concat(annot)
annot.head()

,barcode,is_cell,contig_id,high_confidence,length,chain,v_gene,d_gene,j_gene,c_gene,full_length,productive,cdr3,cdr3_nt,reads,umis,raw_clonotype_id,raw_consensus_id
0,UK-CIC10690366+UK-CIC10690558_AAACCTGAGGCATTGG-1,True,UK-CIC10690366+UK-CIC10690558_AAACCTGAGGCATTGG...,True,667,IGH,IGHV4-34,None,IGHJ4,IGHG1,True,True,CARGIPSVTMVLVVIGMHYYFDSW,TGTGCGAGAGGAATCCCGAGTGTGACTATGGTACTGGTGGTTATTG...,2625,66,clonotype204,clonotype204_consensus_1
1,UK-CIC10690366+UK-CIC10690558_AAACCTGAGGCATTGG-1,True,UK-CIC10690366+UK-CIC10690558_AAACCTGAGGCATTGG...,True,633,IGL,IGLV3-19,None,IGLJ2,IGLC2,True,True,CSSRDGSGDHVLF,TGTAGCTCCCGGGATGGTAGTGGTGACCACGTGCTATTC,15618,179,clonotype204,clonotype204_consensus_2
2,UK-CIC10690366+UK-CIC10690558_AAACCTGCAACTGGCC-1,True,UK-CIC10690366+UK-CIC10690558_AAACCTGCAACTGGCC...,True,548,IGH,IGHV4-34,IGHD3-10,IGHJ4,IGHM,True,True,CARMAYYGSGSYLPLLTDYW,TGTGCGAGGATGGCTTACTATGGTTCGGGGAGTTATCTGCCACTCC...,349,8,clonotype205,clonotype205_consensus_1
3,UK-CIC10690366+UK-CIC10690558_AAACCTGCAACTGGCC-1,True,UK-CIC10690366+UK-CIC10690558_AAACCTGCAACTGGCC...,True,550,IGK,IGKV1D-39,None,IGKJ1,IGKC,True,True,CQQSYSTPRTF,TGTCAACAGAGTTACAGTACCCCTCGGACGTTC,623,5,clonotype205,clonotype205_consensus_2
4,UK-CIC10690366+UK-CIC10690558_AAACCTGCAAGGACTG-1,True,UK-CIC10690366+UK-CIC10690558_AAACCTGCAAGGACTG...,True,622,IGL,IGLV3-27,None,IGLJ3,IGLC2,True,True,CYSTTDNTGVF,TGTTACTCTACGACTGACAATACCGGGGTGTTC,1517,14,clonotype3,clonotype3_consensus_1


In [11]:
os.chdir('/lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal')
if not os.path.exists('processed/BCR'):
    os.makedirs('processed/BCR')

In [12]:
for ind in individuals:
    tmp_df = annot[annot['barcode'].isin(individuals[ind])]
    tmp_seq = {}
    for cid in tmp_df['contig_id']:
        tmp_seq[cid] = seqs[cid]
    if not os.path.exists('processed/BCR/'+ind):
        os.makedirs('processed/BCR/'+ind)
    tmp_df.to_csv('processed/BCR/'+ind+'/all_contig_annotations.csv', index = False)
    out_fasta = 'processed/BCR/'+ind+'/all_contig.fasta'
    fh1 = open(out_fasta, 'w')
    fh1.close()
    out = ''
    for l in tmp_seq:
        out = '>' + l + '\n' + tmp_seq[l] + '\n'
        Write_output(out, out_fasta)

### run the wrapper
```bash
module load ISG/singularity
cd /lustre/scratch117/cellgen/team297/kt16/COVID_imperial_renal/jobs
# this runs the sc-dandelion singularity container script
bsub < prep_bcr.sh
```